In [18]:
import cv2
import imutils
import numpy as np

In [19]:
def get_corner_by_contour(img):
    imcopy=img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    _, img = cv2.threshold(gray, 126, 255, cv2.THRESH_BINARY)
    
    # Apply erotion to reduce white noise
    kernel = np.ones((2, 2), np.uint8)
    img = cv2.erode(img,kernel,iterations=1)
    
    # Find contours
    contours = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours=imutils.grab_contours(contours)
    # Sort contours by area (largest first)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

    cimg=cv2.drawContours(imcopy,contours,0,(0,0,255),4)
    
    pts=None
    # Loop through contours and look for a 4-point polygon
    for cnt in contours:
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
    
        if len(approx) == 4:
            # Found the document contour
            # cv2.drawContours(img, [approx], -1, (0, 255, 0), 4)
    
            # Extract the corner points
            pts = approx.reshape(4, 2)
            print("Corner Points:")
            print(pts)
            break
    
    return pts

In [20]:
def sort_points(pts):
    pts = np.array(pts, dtype="float32")
    rect = np.zeros((4, 2), dtype="float32")

    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]     # top-left
    rect[2] = pts[np.argmax(s)]     # bottom-right

    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left

    return rect

In [21]:
def alignImages(img,pts):
 
 # Destination A4 size (2480x3508 for 300 DPI)
    width, height = 2480,3508
    dst_pts = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype=np.float32)
    
    # Convert images to grayscale
    imGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find homography
    h, _ = cv2.findHomography(pts, dst_pts, cv2.RANSAC)
 
  # Use homography
    channels = 3
    imReg = cv2.warpPerspective(img, h, (width, height))
    
    return imReg

In [22]:
img = cv2.resize(cv2.imread("test1.jpeg", cv2.IMREAD_COLOR),(400,600))
orig = img.copy()
# selecting corner points
cv2.namedWindow("camscanner")
cv2.imshow("camscanner", img)
points=get_corner_by_contour(img)
imReg = alignImages(orig, sort_points(points))
out_img=cv2.resize(imReg,(400,600))
cv2.imwrite("output.jpg",out_img) # Write aligned image to disk.
cv2.imshow("preview",out_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Corner Points:
[[ 97  77]
 [ 30 511]
 [349 526]
 [312  77]]
